In [1]:
from transactions import CotsETL

In [2]:
# Initiate CotsETL 
cots = CotsETL()

In [3]:
# Get database 
df = cots.cotsExtract()

In [4]:
# Transform database 
df_clean = cots.cotsTransform(df)

C:\Users\malon\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\malon\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\malon\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [5]:
df_clean[df_clean['bpid']=='99954']

,contract_id,bpid,signed_date,terminated_date,duration_years_max,duration_years_base,duration_years_actual,signing_org,first_season,last_update
33,83,99954,2020-01-10,None,0.0,1.0,0.0,MIN,2020.0,2020-05-28 19:12:06.420614
34,85,99954,2019-01-11,None,1.0,1.0,1.0,MIN,2019.0,2020-05-28 19:12:06.420614
35,87,99954,2018-03-01,None,1.0,1.0,1.0,MIN,2018.0,2020-05-28 19:12:06.420614
36,89,99954,2017-03-01,None,1.0,1.0,1.0,MIN,2017.0,2020-05-28 19:12:06.420614
37,91,99954,2016-03-02,None,1.0,1.0,1.0,MIN,2016.0,2020-05-28 19:12:06.420614


In [7]:
def batch_load(df, schema, table, con):
    """
    Function to load data frame into PostgreSQL in batches. 
    """ 
    # List columns of data frame 
    df_columns = list(df)
    columns = ",".join(df_columns)

    # Delete all items from the current table
    delete_stmt = "DELETE FROM public.temp_contracts;"

    # Set values and insert statement 
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
    insert_stmt = "INSERT INTO public.temp_contracts({}) {}".format(columns, values)

    # Create a cursor and execute the delete and insert statements 
    cur = con.cursor()
    cur.execute(delete_stmt)
    psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    con.commit()

    """
    Update euston.contracts with data from temp_contracts. 
    """
    # Update statement 
    update_stmt = """UPDATE {}.{} 
                    SET 
                        contract_id = tc.contract_id 
                        , bpid = tc.bpid 
                        , signed_date = tc.signed_date 
                        , terminated_date = tc.terminated_date 
                        , duration_years_max = tc.duration_years_max 
                        , duration_years_base  = tc.duration_years_base
                        , duration_years_actual = tc.duration_years_actual 
                        , signing_org = tc.signing_org 
                        , first_season = tc.first_season 
                        , last_update = tc.last_update 
                    FROM 
                        public.temp_contracts tc
                    WHERE 
                        {}.{}.contract_id = tc.contract_id 
                        and {}.{}.bpid = tc.bpid; 
                """.format(schema, table, schema, table, schema, table)

    # Execute the update statement
    cur.execute(update_stmt)

    # Commit and lose connection
    con.commit()
    cur.close()

In [8]:
def cotsLoad(df, schema, table):
    """
    Function that takes a single argument, a data frame, and connects to BP PostgreSQL database
    to load the transformed Cots Contracts data. 
    Requires the same txt file with the database connection parameters used in cotsExtract(), with the difference
    that the database is different. 
    """

    # Load txt file with connection strings, create the connection string 
    f = open("con1_con.txt", "r")
    ls = [x.strip() for x in f]

    # Connection string requires that the list be in the following order: 
    # ls[3]: user 
    # ls[2]: password
    # ls[1]: hostname 
    # ls[0]: database 
    host, port = ls[1].split(':')
    conn = psycopg2.connect(dbname="cage", user=ls[3], password=ls[2], host=host, port=port, \
            options = f'-c search_path={schema}',)

    batch_load(df, schema, table, conn)

    print("Database upload succesful.")

In [10]:
import psycopg2

In [11]:
cotsLoad(df_clean, schema="euston", table="contracts")

UniqueViolation: duplicate key value violates unique constraint "contract_un"
DETAIL:  Key (bpid, signed_date)=(99954, 2020-01-10) already exists.


In [13]:
df_clean[df_clean['bpid']=='16637']

,contract_id,bpid,signed_date,terminated_date,duration_years_max,duration_years_base,duration_years_actual,signing_org,first_season,last_update
21999,56624,16637,2016-06-03,None,0.0,1.0,0.0,SLN,2016.0,2020-05-28 19:12:06.420614
22000,56626,16637,2014-10-21,None,1.0,1.0,1.0,PHI,2015.0,2020-05-28 19:12:06.420614
22001,56629,16637,2014-07-11,None,1.0,1.0,1.0,TEX,2014.0,2020-05-28 19:12:06.420614
22002,56630,16637,2014-07-08,None,0.0,1.0,0.0,HOU,2014.0,2020-05-28 19:12:06.420614
22003,56633,16637,2014-01-01,None,0.0,1.0,0.0,LAA,2014.0,2020-05-28 19:12:06.420614
22004,56634,16637,2013-01-15,None,1.0,1.0,1.0,LAA,2013.0,2020-05-28 19:12:06.420614
22005,56636,16637,2011-12-12,None,1.0,1.0,1.0,LAA,2012.0,2020-05-28 19:12:06.420614
22006,56639,16637,2011-01-01,None,1.0,1.0,1.0,LAA,2011.0,2020-05-28 19:12:06.420614
22007,56640,16637,2009-01-01,None,2.0,2.0,2.0,LAA,2009.0,2020-05-28 19:12:06.420614
22008,56641,16637,2008-01-01,None,1.0,1.0,1.0,LAA,2008.0,2020-05-28 19:12:06.420614
